In [1]:
import os
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

DATA_DIR = "./data/kaggle-wood-defects"

def count_yolo_classes(labels_dir):
    class_counts = Counter()
    n_bbox_counts = Counter()
    
    # Iterate over all label files
    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):  # Ensure only text files are read
            file_path = os.path.join(labels_dir, filename)
            with open(file_path, "r") as file:
                lines = file.readlines()
                n_bbox_counts[len(lines)] += 1
                if not lines:
                    class_counts["No labels"] += 1  # Count empty files
                else:
                    for line in lines:
                        parts = line.strip().split()
                        if parts:
                            class_id = parts[0]  # First element is class ID
                            class_counts[class_id] += 1
    
    return class_counts, n_bbox_counts


labels_directory = f"{DATA_DIR}/bboxes"  
class_counts, n_bbox_counts = count_yolo_classes(labels_directory)

labelcounts = pd.Series(class_counts, name="Occurances").sort_index()
bbox_per_img = pd.Series(n_bbox_counts, name="N_BBoxes").sort_index()


FileNotFoundError: [Errno 2] No such file or directory: './data/kaggle-wood-defects/bboxes'

In [ ]:
df_bbox = bbox_per_img.to_frame()
df_bbox.plot.bar(width=1, title="Number of Defect Boxes Per Image")

In [ ]:
labelcounts.plot.bar(title="Label Counts")

## Images

In [15]:
DATA_DIR = "./data/kaggle-wood-defects"
from PIL import Image
from dataclasses import dataclass

@dataclass
class BoundingBoxDetails:
    x: int
    y: int
    width: int
    height: int


def get_bounding_box_info(image_id):
    file_path = os.path.join(DATA_DIR, "labels-yolo/") + str(image_id) + ".txt"
    with open(file_path, 'r') as file:
        print(file.readlines()[0].strip())

def draw_bboxes(image_id):
    img_dir = os.path.join(DATA_DIR, "images/") + str(image_id) + ".jpg"
    img = Image.open(img_dir)
    img.show()

# draw_bboxes(99100007)
get_bounding_box_info(99100007)

1 0.49875 0.6279296875 0.0425 0.09765625
